In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, Add, Activation, Concatenate, Input
from tensorflow.keras.models import Model

2023-12-14 11:57:14.224864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 11:57:14.773461: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-12-14 11:57:14.779108: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-12-14 11:57:15.091568: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.


In [2]:
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

base_dir = './dataset'
low_res_dir = os.path.join(base_dir, '480p')
high_res_dir = os.path.join(base_dir, '1080p')

filenames = os.listdir(low_res_dir)
filenames = os.listdir(high_res_dir)

low_res_paths = [os.path.join(low_res_dir, f) for f in filenames]
high_res_paths = [os.path.join(high_res_dir, f) for f in filenames]

In [3]:
print(len(low_res_paths))
print(len(high_res_paths))

5479
5479


In [4]:
def load_and_preprocess_image(low_res_path, high_res_path):
    # Load images
    low_res_img = tf.io.read_file(low_res_path)
    low_res_img = tf.image.decode_png(low_res_img, channels=3)

    high_res_img = tf.io.read_file(high_res_path)
    high_res_img = tf.image.decode_png(high_res_img, channels=3)

    # Ensure the images have a defined shape
    low_res_img.set_shape([None, None, 3])
    high_res_img.set_shape([None, None, 3])

    low_res_img = tf.image.resize(low_res_img, [int(1080/2), int(1920/2)])
    high_res_img = tf.image.resize(high_res_img, [1080, 1920])

    return low_res_img, high_res_img


dataset = tf.data.Dataset.from_tensor_slices((low_res_paths, high_res_paths))
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# # Shuffle, batch, and repeat
dataset = dataset.shuffle(buffer_size=100).batch(25).repeat()
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

2023-12-14 11:57:21.351045: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 11:57:21.375131: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
for low_res_img, high_res_img in dataset.take(1):
    print(low_res_img.shape, high_res_img.shape)

(25, 540, 960, 3) (25, 1080, 1920, 3)


In [6]:
# Previous model
# def create_srcnn():
#     model = models.Sequential()

#     # First convolutional layer with ReLU activation
#     model.add(layers.Conv2D(64, (9, 9), padding='same', activation='relu', input_shape=(None, None, 3)))

#     # Second convolutional layer
#     model.add(layers.Conv2D(32, (1, 1), padding='same', activation='relu'))

#     # Third convolutional layer
#     model.add(layers.Conv2D(3, (3, 3), padding='same'))

#     # Compile the model
#     model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

#     return model

# srcnn_model = create_srcnn()
# srcnn_model.summary()

In [7]:
def create_enhanced_srcnn():
    model = models.Sequential()

    # First convolutional layer
    model.add(layers.Conv2D(64, (9, 9), padding='same', activation='relu', input_shape=(None, None, 3)))
    model.add(layers.BatchNormalization())

    # Additional convolutional layers for deeper network
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())

    # Upsampling layer
    model.add(layers.UpSampling2D())

    # More convolutional layers
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(3, (3, 3), padding='same'))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    return model

enhanced_srcnn_model = create_enhanced_srcnn()
enhanced_srcnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, None, None, 64)    15616     
                                                                 
 batch_normalization (BatchN  (None, None, None, 64)   256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 64)    36928     
                                                                 
 batch_normalization_1 (Batc  (None, None, None, 64)   256       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, None, None, 64)    36928     
                                                                 
 batch_normalization_2 (Batc  (None, None, None, 64)   2

In [8]:
# Simplified Real-ESRGAN
# import tensorflow as tf
# from tensorflow.keras.layers import Conv2D, Add, Activation, Concatenate, Lambda
# from tensorflow.keras.models import Model

# def dense_block(x, filters, kernel_size=3):
#     """ A basic dense block used in RRDB. """
#     for _ in range(4):  # dense blocks consists of 4 Conv2D
#         y = Conv2D(filters, kernel_size, padding='same', activation='relu')(x)
#         x = Concatenate()([x, y])
#     return x

# def rrdb_block(x, filters):
#     """ Residual-in-Residual Dense Block (RRDB). """
#     residual = Conv2D(filters, (1, 1), padding='same')(x)
#     for _ in range(2):  # RRDB consists of 3 dense blocks
#         x = dense_block(x, filters)
#     # Use a 1x1 convolution to match the channel dimensions
#     x = Conv2D(filters, (1, 1), padding='same')(x)
#     return Add()([residual, x])

# def build_rrdb_model(input_shape=(None, None, 3), filters=64):
#     """ Building a model based on RRDB blocks. """
#     inputs = tf.keras.Input(shape=input_shape)
#     x = inputs
#     for _ in range(1):  # Number of RRDB blocks can be adjusted
#         x = rrdb_block(x, filters)
#     outputs = Conv2D(3, (3, 3), padding='same')(x)  # Assuming RGB output
#     model = Model(inputs, outputs)
#     model.compile(optimizer='adam', loss='mean_squared_error')
#     return model

# # Create the RRDB model
# rrdb_model = build_rrdb_model()

# # Print the model summary
# # rrdb_model.summary()

In [ ]:
enhanced_srcnn_model.fit(dataset, epochs=1, steps_per_epoch=len(low_res_paths) // 10)

 33/547 [>.............................] - ETA: 2:02:31 - loss: 13042.2510 - accuracy: 0.5936

In [ ]:
from datetime import datetime

models_bucket_name = 'up-scaling-prod-sage-maker'
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
model_file_name = f"model_{timestamp}.keras"
enhanced_srcnn_model.save(model_file_name)
s3_client.upload_file(model_file_name, models_bucket_name, model_file_name)

In [ ]:
# LOAD model
import tensorflow as tf
models_bucket_name = 'up-scaling-prod-sage-maker'
stored_model = 'model_2023-11-30_20-05.keras' # to change
s3_client.download_file(models_bucket_name, stored_model, stored_model)

srcnn_model = tf.keras.models.load_model(stored_model)

In [ ]:
for i in range(1):
    model_prediction = rrdb_model.predict(np.stack([test_low_image[i]]))
    predicted = np.clip(np.around(model_prediction), 0, 255).astype(np.uint8)[0]
    plot_images(test_high_image[i],test_low_image[i],predicted)

In [ ]:
import cv2
import subprocess
import os

output_file = 'output.mp4'

frames_predicted = []
frames_low = []
frames_high = []

for i in range(100):
    model_prediction = rrdb_model.predict(np.stack([test_low_image[i]]))
    predicted = np.clip(np.around(model_prediction), 0, 255).astype(np.uint8)[0]
    frames_predicted.append(predicted)
    frames_low.append(test_low_image[i])
    frames_high.append(test_high_image[i])
    # plot_images(test_high_image[i],test_low_image[i],predicted)

# Function to save frames as images
def save_frames_as_images(frames, output_folder='frames', format='png'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for i, frame in enumerate(frames):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imwrite(os.path.join(output_folder, f'frame_{i:04d}.{format}'), rgb_frame)

# Save frames as images
save_frames_as_images(frames_predicted, 'frames_predicted')
print('saved_frames_as_images frames_predicted')
save_frames_as_images(frames_low, 'frames_low')
print('saved_frames_as_images frames_low')
save_frames_as_images(frames_high, 'frames_high')
print('saved_frames_as_images frames_high')

# Function to merge images into a video using FFmpeg
def create_video_from_images(image_folder, output_video_file, fps=30):
    cmd = [
        'ffmpeg',
        '-y',  # Automatically overwrite existing files
        '-framerate', str(fps),
        '-i', os.path.join(image_folder, 'frame_%04d.png'),
        '-s', '1920x1080',  # Set resolution to 1080p
        '-sws_flags', 'lanczos',  # High-quality scaling algorithm
        '-c:v', 'libx264',
        '-pix_fmt', 'yuv420p',
        '-b:v', '5000k',  # Set bitrate
        '-crf', '20',  # Lower CRF for better quality
        output_video_file
    ]
    subprocess.run(cmd, check=True)

# Create video from images
create_video_from_images('frames_predicted', 'output_video_predicted.mp4')
print('created_video_from_images output_video_predicted')
create_video_from_images('frames_low', 'output_video_low.mp4')
print('created_video_from_images output_video_low')
create_video_from_images('frames_high', 'output_video_high.mp4')
print('created_video_from_images output_video_high')
